# Module 4: GitHub Copilot Spaces for Testing Workflows

Functionality:-

Copilot Spaces allow you to create reusable prompt templates for testing workflows. Custom templates help maintain consistency across test generation, bug reporting, and code analysis. You can save templates and reuse them across different functions and projects.

In [ ]:
# USE: lab_booking_system.py

### Task 1: Understand code functionality

##### Prompt: "Explain what each function in lab_booking_system.py does and list their input parameters and return values"

What happens: Copilot analyzes all functions and provides overview of booking system functionality

Answer:- 

lab_booking_system.py — 4 simple functions:

check_availability(date, room_type) -> dict (requires both params; returns {"available": True, "rooms": 5} or error).
create_booking(guest_name, room_type, checkin, checkout) -> dict (validates presence of params; returns fixed booking_id on success).
cancel_booking(booking_id) -> dict (requires id; returns cancelled status without existence check).
get_booking_details(booking_id) -> dict (requires id; returns static booking info).
Notes: implementations are placeholders — no date validation, no availability/DB lookups, no duplicate/exists checks, static responses.
notebook (module_4_lab.ipynb) — test-workflow tasks:

Task 1: explain functions (done).
Task 2: identify missing validations (date format, room type, existence checks).
Task 3: generate pytest suite for check_availability (valid/invalid/boundary).
Task 4: review create_booking for bugs (missing date order, duplicate bookings).
Task 5: produce test data sets for create_booking.
Task 6: generate API test scenarios for get_booking_details.
Task 7: create structured bug report for cancel_booking missing existence check.
Task 8: generate negative tests for check_availability.
Task 9: suggest code improvements (validation, error codes, logging).
Task 10: create test execution plan (sequence, dependencies, priorities).

### Task 2: Identify missing validations
#### Prompt: "Analyze lab_booking_system.py and identify which validations are missing in each function"

What happens: Copilot reviews code logic and lists validation gaps like date format checks and room type validation

Answer:- 

Summary — missing validations by function (concise)

lab_booking_system.check_availability

Missing: input type checks (date must be parseable), date format/parse & timezone handling, reject past dates, validate room_type against allowed values, range/format for date (single date vs date-range), availability lookup (currently returns static), consistent error schema (code/message), sanitize inputs.
lab_booking_system.create_booking

Missing: guest_name validation (non-empty, maxlength, allowed chars), room_type enum check, parse/validate checkin and checkout (format, timezone), ensure checkin < checkout and not in past, booking duration limits, availability check for requested dates, reject overlapping/duplicate bookings, generate unique booking_id, input-type validation, consistent error codes, transactional persistence and concurrency handling.
lab_booking_system.cancel_booking

Missing: verify booking exists before cancelling, check current booking status (e.g., already cancelled), auth/permission check, id format validation, idempotency behavior, business rules (cancellation window, penalties), consistent error response when not found.
lab_booking_system.get_booking_details

Missing: booking_id validation, existence check (return 404-style error if not found), auth/permission check, consistent response schema and error codes, handling malformed IDs.

### Task 3: Generate complete test suite
#### Prompt: "Generate pytest test suite for check_availability function covering valid dates, invalid dates, missing parameters, and boundary cases"

What happens: Copilot creates comprehensive test file with multiple test functions

Answer:-

import datetime
import pytest

from module_4.lab_booking_system import check_availability

def test_check_availability_valid_date_and_room():
    res = check_availability("2025-11-11", "Deluxe")
    assert isinstance(res, dict)
    assert res.get("available") is True
    assert isinstance(res.get("rooms"), int)

@pytest.mark.parametrize("date,room_type", [
    (None, "Deluxe"),          # missing date
    ("", "Deluxe"),            # empty date
    ("2025-11-11", None),      # missing room_type
    ("2025-11-11", ""),        # empty room_type
    (None, None),              # both missing
    ("", ""),                  # both empty
])
def test_check_availability_missing_parameters(date, room_type):
    res = check_availability(date, room_type)
    # current implementation returns an error dict when params missing
    assert isinstance(res, dict)
    assert res.get("available") is False
    assert "error" in res

def test_check_availability_invalid_date_format_is_not_validated():
    # current implementation does not parse/validate date strings,
    # so an arbitrary string is treated as present -> available True
    res = check_availability("not-a-date", "Standard")
    assert isinstance(res, dict)
    assert res.get("available") is True
    assert "rooms" in res

def test_check_availability_boundary_dates_today_and_past():
    today = datetime.date.today().isoformat()
    yesterday = (datetime.date.today() - datetime.timedelta(days=1)).isoformat()

    res_today = check_availability(today, "Standard")
    res_yesterday = check_availability(yesterday, "Standard")

    # current implementation does not reject past dates
    assert res_today.get("available") is True
    assert res_yesterday.get("available") is True

### Task 4: Find potential bugs
#### Prompt: "Review create_booking function and identify potential bugs or issues that could cause problems in production"

What happens: Copilot analyzes code and points out missing date validation and no duplicate booking check

Answer:- 

Critical: current create_booking has a bare if not checkin or not checkout: with no body — this causes a SyntaxError on import.
Key functional/validation issues

Syntax error (empty if) — module won't import/run.
Missing date validation: no parsing/format checks for checkin/checkout.
No date logic: not checking checkin < checkout or rejecting past dates.
No availability check — always returns confirmed.
Hard-coded booking_id (BK12345) — not unique.
No persistence / duplicate/overlap checks — allows overlapping bookings.
Inconsistent error/response schema (sometimes {"error":...}, sometimes booking dict).
No room_type validation against allowed values.
No guest_name validation (empty, length, characters).
No logging, no exceptions, no error codes for callers to handle.
No concurrency/transaction handling (race conditions on booking).
No auth/permission checks or ownership for cancel/get operations.

### Task 5: Create test data sets
#### Prompt: "Generate 5 test data sets for create_booking function including valid bookings, invalid guest names, missing dates, and edge cases"

What happens: Copilot produces dictionary with varied test inputs covering different scenarios

Answer:-

Below are 5 concise test data sets you can use in pytest (parametrize) for create_booking. Dates are generated relative to today.

##### ...existing code...
import datetime

today = datetime.date.today()
test_datasets = [
    {
        "id": "valid_future_booking",
        "input": {
            "guest_name": "Alice Smith",
            "room_type": "Deluxe",
            "checkin": (today + datetime.timedelta(days=1)).isoformat(),
            "checkout": (today + datetime.timedelta(days=3)).isoformat(),
        },
        "expected": {"ok": True, "status": "confirmed", "booking_id_present": True},
    },
    {
        "id": "missing_dates",
        "input": {
            "guest_name": "Bob Jones",
            "room_type": "Standard",
            "checkin": None,
            "checkout": None,
        },
        "expected": {"ok": False, "error_contains": "check-in", "error_type": "missing_dates"},
    },
    {
        "id": "invalid_guest_empty",
        "input": {
            "guest_name": "",
            "room_type": "Standard",
            "checkin": (today + datetime.timedelta(days=5)).isoformat(),
            "checkout": (today + datetime.timedelta(days=7)).isoformat(),
        },
        "expected": {"ok": False, "error_contains": "Guest name", "error_type": "invalid_guest"},
    },
    {
        "id": "checkin_after_checkout",
        "input": {
            "guest_name": "Carol",
            "room_type": "Suite",
            "checkin": (today + datetime.timedelta(days=10)).isoformat(),
            "checkout": (today + datetime.timedelta(days=5)).isoformat(),
        },
        "expected": {"ok": False, "error_contains": "Check-in must be before check-out", "error_type": "date_order"},
    },
    {
        "id": "malicious_guest_name",
        "input": {
            "guest_name": "Robert'); DROP TABLE bookings;--",
            "room_type": "Standard",
            "checkin": (today + datetime.timedelta(days=2)).isoformat(),
            "checkout": (today + datetime.timedelta(days=4)).isoformat(),
        },
        "expected": {"ok": False, "error_contains": "invalid", "error_type": "sanitization_or_validation"},
    },
]
##### ...existing code...

### Task 6: Generate API test cases
#### Prompt: "Create API test scenarios for get_booking_details treating it as REST endpoint, include request format and expected responses"
What happens: Copilot generates API test documentation with request/response examples

Answer:-

I'll add a short todo entry for API test scenarios, then open lab_booking_system.py to inspect get_booking_details so scenarios match the function's contract. Expected outcome: file contents and then API test scenarios aligned to its signature.

Starting (1/1) Create API test scenarios for get_booking_details

Read lab_booking_system.py

Progress update — inspected get_booking_details in lab_booking_system.py and now I’ll produce REST-style API test scenarios that map to its behavior (request formats, expected responses, edge cases, and error handling). Below are structured test scenarios ready to use in API tests or for a QA runbook.

API test scenarios for GET /bookings/{booking_id} (get_booking_details)
Assumptions

Endpoint: GET /bookings/{booking_id}
Path param: booking_id (string)
Content-Type: application/json
Responses use JSON bodies matching get_booking_details returns.
Authentication is out-of-scope unless your real service requires it; add auth headers if needed.
For each scenario include: purpose, request, expected HTTP status, expected response body assertions, and notes.

Happy path — valid booking_id
Purpose: Verify booking details are returned for a valid booking_id.
Request:
Method: GET
URL: /bookings/BK12345
Headers: Accept: application/json
Expected:
HTTP 200 OK
JSON body contains:
booking_id == "BK12345"
guest (non-empty string)
room (non-empty string)
status (one of expected statuses: pending|confirmed|cancelled)
Notes:
Current implementation returns guest "John", room "Deluxe", status "confirmed".
Test asserts fields exist and booking_id matches path param.
Error — missing booking_id (empty path)
Purpose: Ensure service returns validation error for missing booking_id.
Request:
Method: GET
URL: /bookings/ (or GET /bookings with no id)
Headers: Accept: application/json
Expected:
HTTP 400 Bad Request (or 404 Not Found if route requires id)
JSON body contains:
error: "Booking ID required"
Notes:
Current function returns {"error": "Booking ID required"} when booking_id falsy; map to 400.
Error — invalid booking_id format
Purpose: Validate handling of malformed booking ids (e.g., SQL meta-chars, too short).
Request:
Method: GET
URL examples:
/bookings/ (empty) — covered above
/bookings/;DROP TABLE bookings; — malicious string
/bookings/123 — possibly invalid format if system expects BK-prefixed ids
Expected:
HTTP 400 Bad Request (preferred) with JSON error message OR HTTP 404 Not Found if not present.
JSON body should not expose stack traces or DB errors.
Notes:
Test should assert the service returns a safe error and does not leak internals.    

### Task 7: Create structured bug report
#### Prompt: "Create detailed bug report for cancel_booking function issue: function does not verify if booking exists before cancellation. Include severity, steps to reproduce, expected vs actual result"

What happens: Copilot generates formatted bug report with all standard fields filled

Answer:-

Bug Report — cancel_booking doesn't verify booking existence before cancellation

Title

cancel_booking does not validate that the booking exists before returning success
ID

BR-0002 (suggested)
Reported

November 11, 2025
Affected file / function

lab_booking_system.py — function cancel_booking(booking_id)
Summary

The cancel_booking function returns a successful cancellation response when given any non-empty booking_id without checking whether the booking actually exists in storage. This allows callers to receive false confirmations for non-existent bookings, leading to data integrity issues, misleading UI, and potential operational problems.
Severity

Medium-High
Rationale: This is a correctness and reliability bug that can mislead users and downstream systems. It may result in incorrect state, support costs, and reconciliation issues. Not a security exploit by itself, but can cause operational and customer-impacting problems.
Priority

P2 (should be addressed in the next sprint or earlier if bookings/cancellations are customer-facing)
Environment / Scope

Present in the in-repo implementation in lab_booking_system.py.
Any code that calls cancel_booking expecting authoritative cancellation will be affected (APIs, UI flows, scheduled jobs).
Root cause

Missing existence check against booking datastore or in-memory registry. The function's logic only checks that booking_id is provided (truthy); it does not verify persistence or that the booking belongs to the requester.
Code snippet (current implementation)

def cancel_booking(booking_id):
    """Cancel existing booking"""
    if not booking_id:
        return {"error": "Booking ID required"}
    return {"booking_id": booking_id, "status": "cancelled"}

### Task 8: Generate negative test cases
#### Prompt: "Generate 5 negative test cases for check_availability function testing invalid inputs, null values, and error conditions"

What happens: Copilot creates test cases specifically for failure scenarios

Answer:-

Here are 5 detailed negative test cases for check_availability (module_4: check_availability(date, room_type)), including inputs, expected results, assertions you can use in unit or API tests.

Missing date (null / None)
Purpose: Verify function rejects missing check date.
Input:
date = None
room_type = "Deluxe"
Expected behavior:
Return a validation error: {"available": False, "error": "Date and room type required"} (or HTTP 400 at API layer)
Assertion examples:
unit: assert result == {"available": False, "error": "Date and room type required"}
API: assert status_code == 400 and response.json()["error"] contains "Date and room type required"
Missing room_type (empty string)
Purpose: Ensure missing room type is treated as error.
Input:
date = "2025-12-01"
room_type = "" (or None)
Expected behavior:
Return validation error same as above: {"available": False, "error": "Date and room type required"}
Assertion examples:
assert result["available"] is False
assert "room type required" in result["error"].lower()
Invalid date format
Purpose: Detect non-ISO/invalid date strings (format validation).
Input:
date = "01-12-2025" or "not-a-date"
room_type = "Deluxe"
Expected behavior:
Return validation error indicating invalid date format, e.g. {"available": False, "error": "Invalid date format"}
(If API: HTTP 400 with JSON error)
Rationale:
Protects downstream date parsing and avoids silent logical errors (treating arbitrary string as truthy).
Assertion examples:
assert result["available"] is False
assert "invalid date" in result["error"].lower()
Date in the past
Purpose: Prevent queries for past dates (business rule).
Input:
date = yesterday's date (e.g., "2025-11-10" when today is 2025-11-11)
room_type = "Standard"
Expected behavior:
Return error: {"available": False, "error": "Date must be today or in the future"}
Assertion examples:
assert result["available"] is False
assert "today or in the future" in result["error"].lower()
Unsupported / malicious room_type input (SQLi / excessively long)
Purpose: Ensure room_type validation and sanitization; defend against malicious input and DoS via extremely long strings.
Input variants:
room_type = "'); DROP TABLE bookings; --"
room_type = "A" * 10000 (very long string)
date = "2025-12-01"
Expected behavior:
Return validation error: {"available": False, "error": "Invalid room type"} or generic input error; do not execute or crash; if too long, return 413 or validation error.
Assertion examples:
assert result["available"] is False
assert "invalid" in result["error"].lower() or "too long" in result["error"].lower()

### Task 9: Suggest code improvements
#### Prompt: "Suggest improvements for lab_booking_system.py to make functions more robust and testable"

What happens: Copilot recommends adding input validation, error codes, and logging


Answer:-

Validate inputs early
Check and parse date (use timezone-aware datetime) and validate room_type against an allowlist.
Return a consistent contract
Use structured responses (dict with keys like success, data, error_code, message) or raise well-documented exceptions instead of mixing strings/dicts.
Use dependency injection for external state
Accept a booking_store / repository object (or pass it into functions) so tests can inject mocks/fakes.
Use explicit types and docstrings
Add type hints and docstrings describing inputs/outputs and error modes for each function.
Handle persistence & existence checks
Verify booking exists before canceling; return 404-style errors at API layer.
Centralize business rules and status transitions
Define allowed status enum and enforce transition matrix in one place (prevent invalid transitions).
Time comparisons with datetimes (not floats)
Use stored created_at (UTC) + timedelta(hours=24) to decide cancellation window—avoid hours_since_order float.
Add logging and audit hooks
Log attempts, results, and reasons for failures (safely, without PII).
Make functions idempotent where appropriate
E.g., canceling an already-cancelled booking should return a clear idempotent result.
Add schema validation & sanitization
Use simple validators (pydantic/cerberus) or helper functions to reject malformed/malicious input.
Replace magic strings with constants
Status names, error codes, and fixed values (like "Booking ID required") should be constants for easy testing.

### Task 10: Create test execution plan
#### Prompt: "Create test execution plan for lab_booking_system.py with test sequence, dependencies, and priority order"

What happens: Copilot generates structured test plan showing which tests to run first and their dependencies

Answer:-

Purpose
A short, actionable plan to execute tests for lab_booking_system.py covering unit, integration, API, and non-functional checks in priority order so you can run and verify correctness efficiently.

Summary (quick)
Highest priority: correctness of cancellation and booking-detail flows (existence, validation, auth).
Approach: start fast unit tests (pure logic + edge cases), then integration tests with an in-memory booking store, then API-level tests (HTTP), then non-functional checks.
Run commands use Python's unittest (or pytest if preferred). Use mocks for external deps.
Test categories & priority (ordered)
Unit tests — Priority: P0 (run first)

Rationale: fast, isolate logic, catch basic defects.
Targets:
check_availability(date, room_type) — validation, invalid inputs.
create_booking(guest_name, room_type, checkin, checkout) — input validation, required fields.
cancel_booking(booking_id) — input validation (empty id).
get_booking_details(booking_id) — missing id behavior.
Expected runtime: < 1 minute.
Dependencies: none (use pure functions or inject a fake store where required).
Unit tests for business rules & boundary cases — Priority: P0

Targets:
Cancellation window (if implemented) — test strict 24h boundary.
Status transition rules (if implemented).
Discount/price & shipping business logic (if included in booking flows).
Run with mocks.
Integration tests (in-memory store) — Priority: P1

Rationale: verify interaction with booking persistence and state transitions without external DB.
Targets:
cancel_booking must verify booking existence and update persisted status.
create_booking should persist and return booking id/status.
get_booking_details should return persisted data.
Setup: provide an in-memory booking_store fixture with create/get/save methods.
Expected runtime: a few minutes.
Dependencies: test helper module for in-memory store.
API-level tests (REST) — Priority: P1/P2

Rationale: validate real HTTP behavior and status codes.
Targets:
GET /bookings/{id} (happy, missing id, not found, auth/permission).
POST /bookings (create flow input validation).
POST /bookings/{id}/cancel (non-existent booking returns 404; existing changes to cancelled).
Setup: start test server or use Flask/FastAPI test client; use staging-like environment.
Dependencies: app runner or mock HTTP layer, auth mocks if needed.
Security & negative tests — Priority: P2

Rationale: validate input sanitization and auth.
Targets: SQLi payloads for room_type, long strings, missing auth, permissions.
Tools: static analyzer + dynamic tests (OWASP ZAP lightly).
Non-functional checks — Priority: P3

Light performance checks (latency for GET booking) and load tests if needed.
Soak tests if memory/resource leaks suspected.
